In [40]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import fft, signal, stats
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict as dd
import os

plt.style.use('seaborn-whitegrid')

In [41]:
# data = {
#     'a1': [-3.93468,
#             2.40285,
#             6.24273,
#             -3.99411,
#             1.76551
#             ],
#  'a2': [6.55216,
#   9.99438,
#   -3.17577,
#   -14.1448,
#   -6.46492],
#  'a3' : [-1.23798,
#   -3.24265,
#   -0.686974,
#   3.84536,
#   5.09623],
#  'a4':[20.3103,
#   8.3132,
#   -4.19382,
#   -5.6586,
#   -9.00136],
#  'load':[50, 50, 50, 50, 50]
#  }

# data = {
#   "a1": {-3.93468,2.40285,6.24273,-3.99411,1.76551,8.0413,-4.01812,4.91148,9.87637,-4.45857,-5.11952,-9.42848,-3.65882,0.166275,0.325032
#   },
#   "a2": {
#     6.55216,9.99438,-3.17577,-14.1448,-6.46492,0.962914,1.07599,-4.4753,1.05646,5.19339,5.6831,5.67469,1.68175,-0.326172,5.88362
#   },
#   "a3": {-1.23798,-3.24265,-0.686974,3.84536,5.09623,-1.54231,7.16725,4.65973,-6.2591,-13.3042,-10.0638,0.432831,7.39364,2.70997,-0.036916
#   },
#   "a4": {20.3103,8.3132,-4.19382,-5.6586,-9.00136,2.54418,5.76429,1.98554,0.918555,-2.54888,0.417422,-2.98862,-5.52874,7.74519,10.6415
#   },

#   "load": [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
#   ]
# }
# # Healthy
data = {
  'a1':{-10.8023,13.9087,6.37438,-15.2817,3.13954,0.677448,-10.5754,-4.03329,1.86867,7.58148},
 'a2':{-5.15393,7.88091,-3.59342,3.73856,6.8219,-3.23441,7.7254,2.57692,-5.0894,6.20596},
 'a3':{6.84189,-2.98886,0.590603,4.00437,-2.92981,-1.72599,-2.18401,1.46843,5.34229,-6.12133},
 'a4':{-3.60985,6.41759,2.61505,1.52184,1.55903,-3.14302,2.56965,2.72891,-1.36563,11.5483},
 "load": [90,90,90,90,90,90,90,90,90,90]
}

# data is for broken 
# Convert to JSON to dataframe
df = pd.DataFrame.from_dict(data, orient='index').T
			

In [42]:
sensor_readings = df.melt(
    id_vars=['load'],
    value_vars=['a1','a2','a3','a4'],
    var_name='sensor',
    value_name='reading'
)

In [43]:
sensor_readings

load sensor    reading
0   90.0     a1   0.677448
1   90.0     a1   1.868670
2   90.0     a1   3.139540
3   90.0     a1   6.374380
4   90.0     a1   7.581480
5   90.0     a1  13.908700
6   90.0     a1 -15.281700
7   90.0     a1 -10.802300
8   90.0     a1 -10.575400
9   90.0     a1  -4.033290
10  90.0     a2   2.576920
11  90.0     a2   3.738560
12  90.0     a2   6.821900
13  90.0     a2   7.725400
14  90.0     a2   7.880910
15  90.0     a2   6.205960
16  90.0     a2  -5.153930
17  90.0     a2  -3.593420
18  90.0     a2  -5.089400
19  90.0     a2  -3.234410
20  90.0     a3   0.590603
21  90.0     a3   1.468430
22  90.0     a3   4.004370
23  90.0     a3   5.342290
24  90.0     a3   6.841890
25  90.0     a3  -2.184010
26  90.0     a3  -2.929810
27  90.0     a3  -6.121330
28  90.0     a3  -2.988860
29  90.0     a3  -1.725990
30  90.0     a4   1.521840
31  90.0     a4   2.728910
32  90.0     a4   2.615050
33  90.0     a4   2.569650
34  90.0     a4   1.559030
35  90.0     a4   6.417590
36  90.0     a4  11.548300
37  90.0     a4  -3.609850
38  90.0     a4  -3.143020
39  90.0     a4  -1.365630

In [44]:
sensor_readings.shape


(40, 3)

In [45]:
data = []
labels = []
for (load,sensor),g in sensor_readings.groupby(['load','sensor']):
    # print("load: {0} for sensor: {1}".format(load,sensor))
    # print("g.reading.values:",g.reading.values)
    vals = g.reading.values
    splits = np.split(vals, range(1000,vals.shape[0],1000))
    # print(load,sensor, splits[:])
    # print("splits.shape:",len(splits[0]))
    for i,s in enumerate(splits):  # except the last one
        # print("i:",i)
        data.append({
            'sensor_a1': int(sensor=='a1'),
            'sensor_a2': int(sensor=='a2'),
            'sensor_a3': int(sensor=='a3'),
            # no need to put a4: if a1-3 are 0, then it's sensor a4
            'load': load,
            'mean': np.mean(s),
            'std': np.std(s),
            'kurt': stats.kurtosis(s),
            'skew': stats.skew(s),
            'moment': stats.moment(s),
        })
        # labels.append(int(state=='b'))  # positive label indicates fault
# print("shape of original data: ", sensor_readings.shape)
# print("shape of data after splitting: ", len(data))
df_data = pd.DataFrame(data)
# print(df_data)
data = df_data.values
data.shape
# print(df_data.values)
# labels = np.array(labels)
# (data.shape, labels.shape)

(4, 9)

In [46]:
sensor_readings

load sensor    reading
0   90.0     a1   0.677448
1   90.0     a1   1.868670
2   90.0     a1   3.139540
3   90.0     a1   6.374380
4   90.0     a1   7.581480
5   90.0     a1  13.908700
6   90.0     a1 -15.281700
7   90.0     a1 -10.802300
8   90.0     a1 -10.575400
9   90.0     a1  -4.033290
10  90.0     a2   2.576920
11  90.0     a2   3.738560
12  90.0     a2   6.821900
13  90.0     a2   7.725400
14  90.0     a2   7.880910
15  90.0     a2   6.205960
16  90.0     a2  -5.153930
17  90.0     a2  -3.593420
18  90.0     a2  -5.089400
19  90.0     a2  -3.234410
20  90.0     a3   0.590603
21  90.0     a3   1.468430
22  90.0     a3   4.004370
23  90.0     a3   5.342290
24  90.0     a3   6.841890
25  90.0     a3  -2.184010
26  90.0     a3  -2.929810
27  90.0     a3  -6.121330
28  90.0     a3  -2.988860
29  90.0     a3  -1.725990
30  90.0     a4   1.521840
31  90.0     a4   2.728910
32  90.0     a4   2.615050
33  90.0     a4   2.569650
34  90.0     a4   1.559030
35  90.0     a4   6.417590
36  90.0     a4  11.548300
37  90.0     a4  -3.609850
38  90.0     a4  -3.143020
39  90.0     a4  -1.365630

In [47]:
df_data

sensor_a1  sensor_a2  sensor_a3  load      mean       std      kurt  \
0          1          0          0  90.0 -0.714247  8.823809 -1.049840   
1          0          1          0  90.0  1.787849  5.208984 -1.683409   
2          0          0          1  90.0  0.229758  3.950954 -1.078330   
3          0          0          0  90.0  2.084187  4.260516  0.130611   

       skew  moment  
0 -0.142996     0.0  
1 -0.201667     0.0  
2  0.220244     0.0  
3  0.725117     0.0

In [48]:
cols_to_scale = ['load','mean','std','kurt','skew','moment']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_data[cols_to_scale] = scaler.fit_transform(df_data[cols_to_scale])

In [49]:
df_data


sensor_a1  sensor_a2  sensor_a3  load      mean       std      kurt  \
0          1          0          0   0.0  0.000000  1.000000  0.349262   
1          0          1          0   0.0  0.894106  0.258171  0.000000   
2          0          0          1   0.0  0.337333  0.000000  0.333557   
3          0          0          0   0.0  1.000000  0.063528  1.000000   

       skew  moment  
0  0.063306     0.0  
1  0.000000     0.0  
2  0.455241     0.0  
3  1.000000     0.0

In [50]:
import tensorflow as tf

In [51]:
model = tf.keras.models.load_model('mdoels\model_700_r.h5')

In [52]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 100)               1000      
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 25)                1275      
                                                                 
 dense_29 (Dense)            (None, 12)                312       
                                                                 
 dense_30 (Dense)            (None, 6)                 78        
                                                                 
 dense_31 (Dense)            (None, 3)                 21        
                                                                 
 dense_32 (Dense)            (None, 1)                

In [53]:
df_data.values[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.34926232, 0.06330601, 0.        ])

In [54]:
prediction = model.predict(df_data.values)

1/1 [==============================] - 0s 218ms/step


In [57]:
prediction[0]

array([0.00077489], dtype=float32)

In [58]:
np.round(prediction[0])

array([0.], dtype=float32)

In [55]:

import numpy as np
y_label = {0:"Healthy",1:"Broken"}
print(y_label[np.max(np.round(prediction))])

Broken
